In [1]:
import pandas as pd
import numpy as np

<hr style='margin-top:31px; border: 2px solid #43091A'>
<div style='text-align: center ; font-size: 25px; color:#43091A'>продажи</div>

 ### столбцы
 date - Документ продажи.Дата
 
 check - Документ продажи.Номер чека ККМ
 
 nom - Номенклатура.Наименование
 
 group - Группа.Наименование
 
 ps - Магазин.Наименование
 
 town - Магазин.Контактная информация.Город
 
 num_goods - Количество товаров
 
 price - Сумма продаж со скидкой без НДС
 
 disc - Сумма продаж со скидкой

## загружаем данные 

In [2]:
sales = pd.read_excel('file/Отчет по продажам январь 2021.xls', skiprows=6, header=None)
sales.columns = ['date', 'check', 'nom', 'group', 'ps', 'town', 'num_goods', 'price', 'disc']

In [3]:
sales.head(5)

,date,check,nom,group,ps,town,num_goods,price,disc
0,01.01.2021 0:26:25,207546.0,"Вода мин. Крымская слабогаз. 1,0л ПЭТ",БЕЗАЛКОГОЛЬНЫЕ НАПИТКИ,АЗС №2 г.Керчь,NaN,1,33.33,40.0
1,01.01.2021 0:26:25,207546.0,"Вода мин. Крымская слабогаз. 1,0л ПЭТ",БЕЗАЛКОГОЛЬНЫЕ НАПИТКИ,АЗС №2 г.Керчь,Керчь г,1,33.33,40.0
2,01.01.2021 0:26:25,207546.0,Кофе АМЕРИКАНО 200 мл АЗС 2 АЗС 4,КОФЕЙНЫЕ НАПИТКИ,АЗС №2 г.Керчь,NaN,1,50.00,60.0
3,01.01.2021 0:26:25,207546.0,Кофе АМЕРИКАНО 200 мл АЗС 2 АЗС 4,КОФЕЙНЫЕ НАПИТКИ,АЗС №2 г.Керчь,Керчь г,1,50.00,60.0
4,01.01.2021 0:28:27,207548.0,Fhilip Morris Compakt Яркий,ТАБАЧНАЯ ПРОДУКЦИЯ,АЗС №2 г.Керчь,NaN,1,95.83,115.0


<hr style='margin:31px 0 33px 0; border: 1px solid #43091A'>

### убираем время из столбца с датой 

In [4]:
sales['date'] = pd.to_datetime(sales['date'], errors='coerce').dt.date
sales.head(3)

,date,check,nom,group,ps,town,num_goods,price,disc
0,2021-01-01,207546.0,"Вода мин. Крымская слабогаз. 1,0л ПЭТ",БЕЗАЛКОГОЛЬНЫЕ НАПИТКИ,АЗС №2 г.Керчь,NaN,1,33.33,40.0
1,2021-01-01,207546.0,"Вода мин. Крымская слабогаз. 1,0л ПЭТ",БЕЗАЛКОГОЛЬНЫЕ НАПИТКИ,АЗС №2 г.Керчь,Керчь г,1,33.33,40.0
2,2021-01-01,207546.0,Кофе АМЕРИКАНО 200 мл АЗС 2 АЗС 4,КОФЕЙНЫЕ НАПИТКИ,АЗС №2 г.Керчь,NaN,1,50.00,60.0


### дубликаты - в столбце 'town' имеется NaN -> удаляем строку 

In [5]:
sales = sales.dropna(subset=['town'])

In [6]:
sales.replace('', np.nan, inplace=True)
sales.head(6)

,date,check,nom,group,ps,town,num_goods,price,disc
1,2021-01-01,207546.0,"Вода мин. Крымская слабогаз. 1,0л ПЭТ",БЕЗАЛКОГОЛЬНЫЕ НАПИТКИ,АЗС №2 г.Керчь,Керчь г,1,33.33,40.0
3,2021-01-01,207546.0,Кофе АМЕРИКАНО 200 мл АЗС 2 АЗС 4,КОФЕЙНЫЕ НАПИТКИ,АЗС №2 г.Керчь,Керчь г,1,50.00,60.0
5,2021-01-01,207548.0,Fhilip Morris Compakt Яркий,ТАБАЧНАЯ ПРОДУКЦИЯ,АЗС №2 г.Керчь,Керчь г,1,95.83,115.0
7,2021-01-01,207550.0,Вода 5 л детс.,БЕЗАЛКОГОЛЬНЫЕ НАПИТКИ,АЗС №2 г.Керчь,Керчь г,1,63.64,70.0
9,2021-01-01,207550.0,Вода Aquita 5 л,БЕЗАЛКОГОЛЬНЫЕ НАПИТКИ,АЗС №2 г.Керчь,Керчь г,1,58.33,70.0
11,2021-01-01,207552.0,Davidoff Classic(135),ТАБАЧНАЯ ПРОДУКЦИЯ,АЗС №2 г.Керчь,Керчь г,1,143.33,172.0


## заполняем пустые значения 
-  столбец 'check' значением Nan
-  столбцы 'disc' 'price' - нулями 

In [8]:
sales['check'].replace(to_replace=[None], value=np.nan, inplace=True)
sales['disc'].fillna(0.0, inplace=True)
sales['price'].fillna(0.0, inplace=True)

### sum( дата+номенклатура+АЗС ) 

In [9]:
amount = pd.DataFrame(sales.groupby(by=[sales['date'],sales['ps'],sales['nom']])['num_goods', 'disc'].sum().reset_index())
# сумма продаж со скидкой 
amount.columns = ['Дата', 'Магазин', 'Товар','Количество', 'Сумма продаж']
amount.head(9)

/var/folders/1m/r8sjkd3n21s0fjc__2nczm000000gn/T/ipykernel_8076/1224971307.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  amount = pd.DataFrame(sales.groupby(by=[sales['date'],sales['ps'],sales['nom']])['num_goods', 'disc'].sum().reset_index())


,Дата,Магазин,Товар,Количество,Сумма продаж
0,2021-01-01,АЗС № 1 Героев Сталинграда 8А,Вода TASSAY 1 л в ассортименте,1,50.0
1,2021-01-01,АЗС № 1 Героев Сталинграда 8А,Вода Крымская столовая 6 л негаз ПЭТ,1,100.0
2,2021-01-01,АЗС № 1 Героев Сталинграда 8А,"Вода мин. Боржоми 0,75 л ПЭТ",2,250.0
3,2021-01-01,АЗС № 1 Героев Сталинграда 8А,"КРЫМ Напиток Лимонад 1,0л ПЭТ",5,325.0
4,2021-01-01,АЗС № 1 Героев Сталинграда 8А,"Крымская Вода мин. газ 2,0 ПЭТ",1,50.0
5,2021-01-01,АЗС № 1 Героев Сталинграда 8А,"Крымская Вода питьевая н/газ 1,0л ПЭТ",1,40.0
6,2021-01-01,АЗС № 1 Героев Сталинграда 8А,Нагреваемые палочки (стики) HEETS Bronze Lable,1,160.0
7,2021-01-01,АЗС № 1 Героев Сталинграда 8А,"Напиток б/а тониз. энергет. газ Ягуар Фри 0,5 ...",1,60.0
8,2021-01-01,АЗС № 1 Героев Сталинграда 8А,Сникерс Супер 95 г,2,140.0


### удаляем последнюю строку – имеющую итоговую строку 

In [10]:
sales['data_date'] = sales.loc[sales.index[-1], 'date']

In [11]:
sales.tail(3)

,date,check,nom,group,ps,town,num_goods,price,disc,data_date
20439,2021-01-31,1089.0,Напиток б/в энер. газ Генезис Фиолетовая звезд...,БЕЗАЛКОГОЛЬНЫЕ НАПИТКИ,АЗС №2 г.Керчь,Керчь г,2,125.00,150.0,2021-01-31
20441,2021-01-31,1089.0,Печенье Яшкино сэндвич сливочный 182 гр,FOOD,АЗС №2 г.Керчь,Керчь г,1,37.50,45.0,2021-01-31
20442,2021-01-31,147887.0,Масло мотор. 10 w 40 ZIC X5 API SM п/синт. 1 л,NON FOOD,АЗС № 1 Героев Сталинграда 8А,Симферополь г,1,316.67,380.0,2021-01-31


<hr style='margin-top:31px; border: 2px solid #43091A'>
<div style='text-align: center ; font-size: 25px; color:#43091A'>наценки</div>

### столбцы
ps - Магазин

group - Номенклатура, Группа

num - Количество, шт

pсh - Закупочная сумма

retail - Розничная сумма

margin - Сумма наценки

% - Процент наценки

%_m - Процент маржи

In [12]:
margin = pd.read_excel('file/Отчет по наценке январь 2021.xls', skiprows=5, header=None)
margin = margin.drop([1, 3], axis=1)
margin.columns = ['ps','group','num', 'pсh', 'retail', 'margin','%', '%_m']

margin.tail(5)

,ps,group,num,pсh,retail,margin,%,%_m
933,АЗС №3 г.Севастополь,"Т Кофе эспрессо, КОФЕЙНЫЕ НАПИТКИ",3,33.33,150.00,116.67,350.05,77.78
934,АЗС №3 г.Севастополь,"Т Чай в ассорт., ЧАЙНЫЕ НАПИТКИ",5,10.05,100.00,89.95,895.02,89.95
935,АЗС №3 г.Севастополь,"Т Шоколад с молоком, КОФЕЙНЫЕ НАПИТКИ",3,36.78,150.00,113.22,307.83,75.48
936,АЗС №3 г.Севастополь,"Шоколадный батончик Bounty 55г, FOOD",6,162.48,240.00,77.52,47.71,32.30
937,Итого,NaN,14346,1054416.39,1379385.78,324969.39,30.82,23.56
